# Check MinIO

In [1]:
!pip install minio delta-spark==2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 8.9 MB/s eta 0:00:00


In [1]:
from minio import Minio

In [2]:
client = Minio(
    "minio:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)

bucket = "warehouse"
if client.bucket_exists(bucket):
    print(f"{bucket} exists")


warehouse exists


# Init SparkContext

In [3]:
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [4]:
spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()


/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.2-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.2-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file

In [5]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-rdd-demo-2024-07-08 11:35:13.469743>

In [9]:
# test Delta Lake
spark.range(50).write.format("delta").save("s3a://warehouse/deltafile", mode="overwrite")

Py4JJavaError: An error occurred while calling o139.defaultParallelism.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py

# Create RDDs

## By loading dataset

In [6]:
fdd = sc.textFile("s3a://warehouse/testfile.txt")
fdd

s3a://warehouse/testfile.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [7]:
fdd.getNumPartitions()

2

In [9]:
fdd.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 12) (172.20.0.5 executor 12): ExecutorLostFailure (executor 12 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutabl

In [ ]:
fdd = fdd.repartition(10)
fdd.getNumPartitions()

In [ ]:
fdd.count()

In [ ]:
fdd.collect()

In [ ]:
# split lines into words
words = fdd.flatMap(lambda line: line.split())

# count the occurrences of each word
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts.collect()

In [ ]:
# filter and split lines into words
words_2 = fdd.filter(lambda x: "There are" in x).flatMap(lambda line: line.split())

# count the occurrences of each word
word_counts_2 = words_2.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts_2.collect()

In [ ]:
word_counts.join(word_counts_2).collect()

In [ ]:
# create two RDDs with key-value pairs
rdd1 = sc.parallelize([(1, "hello"), (2, "world"), (3, "foo")], 1)
rdd2 = sc.parallelize([(1, "bar"), (2, "baz"), (4, "qux")], 1)

# perform a join on the two RDDs
rdd_joined = rdd1.join(rdd2)

# print the result
rdd_joined.collect()

## By using parallelize

In [ ]:
data = [1, 2, 3, 3]
rdd = sc.parallelize(data, 2)
rdd

In [ ]:
rdd.getNumPartitions()

In [ ]:
rdd.collect()

# RDD operations

## Transformation

* Element-wise transformations
* Transformation filter(): Takes in a function and returns an RDD that only has elements that pass the filter( ) function

In [ ]:
rdd.filter(lambda x: x != 1).collect()

* Element-wise transformations
* Transformation map(): Takes in a function and applies it to each element  in the RDD with the result of the function being the new value of each element in the resulting RDD

In [ ]:
rdd.map(lambda x: x + 1).collect()

In [ ]:
rdd.map(lambda x: [x, x + 5]).collect()

In [ ]:
rdd.flatMap(lambda x: [x, x + 5]).collect()

* Sampling Transformation
* sample() an RDD: We can specify with or without replacement, or the fraction

In [ ]:
rdd.sample(False, 0.5).collect()

* Pseudo Set Operations
* RDDs support many operations of mathematical sets: 
* distinct, union, intersect, subtract
* All expensive except union because they involve shuffling

In [ ]:
rdd = sc.parallelize([1, 2, 3])
other = sc.parallelize([3, 4, 5])

In [ ]:
rdd.union(other).collect()

In [ ]:
rdd.intersection(other).collect()

In [ ]:
rdd.subtract(other).collect()

In [ ]:
rdd.cartesian(other).collect()

## Actions

* fold( )
* takes a function similarly as reduce( ) does, but takes a “zero value” to be used for initial call on each partition
* should be the identity element for the operation
* 0 for +, 1 for *, etc.
* return type the same as RDD elements 


In [ ]:
rdd = sc.parallelize([1, 2, 3, 3])
rdd.fold(0, lambda a, b: a + b)

* reduce( )
* takes a function that operates on two elements of the type in the RDD and returns a new element of the same type
* should be commutative and associative so that it can be computed correctly in parallel

In [ ]:
rdd.reduce(lambda a, b: a + b)

aggregate( )
* we can also supply an initial zero value of the type we want to return
* a 1st function to combine the elements from RDD with the accumulator
* a 2nd function to merge two accumulators given that each node accumulates its own results locally

In [ ]:
seq_op = (lambda acc, value: (acc[0] + value, acc[1] + 1))
comb_op = (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))

sumCount = rdd.aggregate((0, 0), seq_op, comb_op)
sumCount[0], sumCount[1], sumCount[0]/float(sumCount[1])

collect( )
* return the entire RDD’s contents to the driver program

take( )
* returns n elements from the RDD
* attempts to minimize the number of partitions it accesses, so may be biased

top(n)
* return the top n elements of the RDD

count( )
* returns the number of elements in the RDD

In [ ]:
rdd.collect()

In [ ]:
rdd.take(2)

In [ ]:
rdd.takeOrdered(2, key=lambda x: -x)

In [ ]:
rdd.takeSample(False, 1)

In [ ]:
rdd.top(2)

In [ ]:
rdd.count()

In [ ]:
rdd.countByValue()

# Caching RDDs

In [ ]:
lines = sc.textFile("s3a://warehouse/testfile.txt", 4)
lines

Count will cause Spark to
* read data
* sum within partitions
* combine sums in driver

In [ ]:
lines.count()

In [ ]:
paragraphs = lines.filter(lambda x: len(x) > 0)
paragraphs

Count will cause Spark to
* read data (again)
* sum within partitions
* combine sums in driver

In [ ]:
paragraphs.count()

In [ ]:
lines = sc.textFile("s3a://warehouse/testfile.txt", 4)

# save, don't compute
lines.cache()

In [ ]:
paragraphs = lines.filter(lambda x: len(x) > 0)
print(lines.count())

In [ ]:
print(paragraphs.count())

In [ ]:
# remove from cache
lines.unpersist()